<a href="https://colab.research.google.com/github/hargurjeet/Suprise_Housing/blob/main/Suprise_Housing_Case_study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing the data

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder

In [2]:
input_url = 'https://raw.githubusercontent.com/hargurjeet/Suprise_Housing/main/train.csv'
raw_data = pd.read_csv(input_url)
raw_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [4]:
#removing columns with high missing values
pd.set_option('display.max_rows', None)
columns_with_high_missing_values = raw_data.columns[[((raw_data.isnull().sum()/len(raw_data))*100 > 40)]]
columns_with_high_missing_values

/usr/local/lib/python3.8/dist-packages/pandas/core/indexes/base.py:4616: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = getitem(key)


Index(['Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature'], dtype='object')

In [5]:
print(len(raw_data.columns))
raw_data = raw_data.drop(labels=columns_with_high_missing_values, axis=1)
raw_data = raw_data.drop(labels='Id', axis=1)
print(len(raw_data.columns))

81
75


In [6]:
X = raw_data.drop('SalePrice', axis=1)
y = raw_data['SalePrice']

In [7]:
y.shape

(1460,)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Imputation

In [9]:
## seprating numberical and categorical columns
X_train_numberic = X_train.select_dtypes(include=['number'])
X_train_cat = X_train.select_dtypes(include=['object'])

In [10]:
print(X_train_cat.shape, X_train_numberic.shape)

(978, 38) (978, 36)


In [11]:
pd.set_option('display.max_column', None)
X_train_numberic.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
615,85,80.0,8800,6,7,1963,1963,156.0,763,0,173,936,1054,0,0,1054,1,0,1,0,3,1,6,0,1963.0,2,480,120,0,0,0,0,0,0,5,2010
613,20,70.0,8402,5,5,2007,2007,0.0,206,0,914,1120,1120,0,0,1120,0,0,1,0,3,1,6,0,NaN,0,0,0,30,0,0,0,0,0,12,2007
1303,20,73.0,8688,7,5,2005,2005,228.0,0,0,1616,1616,1616,0,0,1616,0,0,2,0,3,1,7,0,2005.0,3,834,208,59,0,0,0,0,0,4,2006
486,20,79.0,10289,5,7,1965,1965,168.0,836,0,237,1073,1073,0,0,1073,1,0,1,1,3,1,6,0,1965.0,2,515,0,0,0,0,0,0,0,6,2007
561,20,77.0,10010,5,5,1974,1975,0.0,1071,123,195,1389,1389,0,0,1389,1,0,1,0,2,1,6,1,1975.0,2,418,240,38,0,0,0,0,0,4,2006


In [12]:
## Observing few column can be classifed as catergorical instead on numberical
cat_cols = ['YearBuilt', 'GarageYrBlt', 'YrSold']
X_train_numberic[cat_cols] = X_train_numberic[cat_cols].astype(object)
X_train_cat = pd.concat([X_train_cat.sort_index(), X_train_numberic[cat_cols].sort_index()],  axis=1)
X_train_numberic = X_train_numberic.drop(cat_cols, axis=1)
print(X_train_cat.shape, X_train_numberic.shape)

(978, 41) (978, 33)


/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


## Imputing numberical columns

In [13]:
X_train_numberic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 978 entries, 615 to 1126
Data columns (total 33 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     978 non-null    int64  
 1   LotFrontage    795 non-null    float64
 2   LotArea        978 non-null    int64  
 3   OverallQual    978 non-null    int64  
 4   OverallCond    978 non-null    int64  
 5   YearRemodAdd   978 non-null    int64  
 6   MasVnrArea     976 non-null    float64
 7   BsmtFinSF1     978 non-null    int64  
 8   BsmtFinSF2     978 non-null    int64  
 9   BsmtUnfSF      978 non-null    int64  
 10  TotalBsmtSF    978 non-null    int64  
 11  1stFlrSF       978 non-null    int64  
 12  2ndFlrSF       978 non-null    int64  
 13  LowQualFinSF   978 non-null    int64  
 14  GrLivArea      978 non-null    int64  
 15  BsmtFullBath   978 non-null    int64  
 16  BsmtHalfBath   978 non-null    int64  
 17  FullBath       978 non-null    int64  
 18  HalfBat

In [14]:
X_train_numberic.isnull().sum()

MSSubClass         0
LotFrontage      183
LotArea            0
OverallQual        0
OverallCond        0
YearRemodAdd       0
MasVnrArea         2
BsmtFinSF1         0
BsmtFinSF2         0
BsmtUnfSF          0
TotalBsmtSF        0
1stFlrSF           0
2ndFlrSF           0
LowQualFinSF       0
GrLivArea          0
BsmtFullBath       0
BsmtHalfBath       0
FullBath           0
HalfBath           0
BedroomAbvGr       0
KitchenAbvGr       0
TotRmsAbvGrd       0
Fireplaces         0
GarageCars         0
GarageArea         0
WoodDeckSF         0
OpenPorchSF        0
EnclosedPorch      0
3SsnPorch          0
ScreenPorch        0
PoolArea           0
MiscVal            0
MoSold             0
dtype: int64

In [15]:
X_train_numberic = X_train_numberic.drop(labels=['LotFrontage','MasVnrArea'], axis=1)

In [16]:
# scaling the data between 0 and 1
scaler = MinMaxScaler()
X_train_numberic_scaled = pd.DataFrame(scaler.fit_transform(X_train_numberic), columns = X_train_numberic.columns)
X_train_numberic_scaled.head()

,MSSubClass,LotArea,OverallQual,OverallCond,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold
0,0.382353,0.035056,0.555556,0.75,0.216667,0.135188,0.000000,0.074058,0.153191,0.165213,0.0,0.0,0.135644,0.333333,0.0,0.333333,0.0,0.375,0.5,0.333333,0.000000,0.50,0.338505,0.140023,0.000000,0.0,0.0,0.0,0.0,0.0,0.363636
1,0.000000,0.033195,0.444444,0.50,0.950000,0.036499,0.000000,0.391267,0.183306,0.180358,0.0,0.0,0.148078,0.000000,0.0,0.333333,0.0,0.375,0.5,0.333333,0.000000,0.00,0.000000,0.000000,0.054845,0.0,0.0,0.0,0.0,0.0,1.000000
2,0.000000,0.034532,0.666667,0.50,0.916667,0.000000,0.000000,0.691781,0.264484,0.294172,0.0,0.0,0.241522,0.000000,0.0,0.666667,0.0,0.375,0.5,0.416667,0.000000,0.75,0.588152,0.242707,0.107861,0.0,0.0,0.0,0.0,0.0,0.272727
3,0.000000,0.042015,0.444444,0.75,0.250000,0.148122,0.000000,0.101455,0.175614,0.169573,0.0,0.0,0.139224,0.333333,0.0,0.333333,0.5,0.375,0.5,0.333333,0.000000,0.50,0.363188,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.454545
4,0.000000,0.040711,0.444444,0.50,0.416667,0.189759,0.109139,0.083476,0.227332,0.242084,0.0,0.0,0.198757,0.333333,0.0,0.333333,0.0,0.250,0.5,0.333333,0.333333,0.50,0.294781,0.280047,0.069470,0.0,0.0,0.0,0.0,0.0,0.272727


## Imputing categorical values

In [17]:
X_train_cat.head()

,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,SaleType,SaleCondition,YearBuilt,GarageYrBlt,YrSold
0,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,Gd,TA,PConc,Gd,TA,No,GLQ,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,Attchd,RFn,TA,TA,Y,WD,Normal,2003,2003.0,2008
1,RL,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,Gable,CompShg,MetalSd,MetalSd,None,TA,TA,CBlock,Gd,TA,Gd,ALQ,Unf,GasA,Ex,Y,SBrkr,TA,Typ,Attchd,RFn,TA,TA,Y,WD,Normal,1976,1976.0,2007
2,RL,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,Gd,TA,PConc,Gd,TA,Mn,GLQ,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,Attchd,RFn,TA,TA,Y,WD,Normal,2001,2001.0,2008
3,RL,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,Gable,CompShg,Wd Sdng,Wd Shng,None,TA,TA,BrkTil,TA,Gd,No,ALQ,Unf,GasA,Gd,Y,SBrkr,Gd,Typ,Detchd,Unf,TA,TA,Y,WD,Abnorml,1915,1998.0,2006
4,RL,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,Gd,TA,PConc,Gd,TA,Av,GLQ,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,Attchd,RFn,TA,TA,Y,WD,Normal,2000,2000.0,2008


In [18]:
X_train_cat = X_train_cat.where(pd.notnull(X_train_cat), None)

In [19]:
imp_median = SimpleImputer(missing_values=None, strategy='most_frequent')
X_train_cat = pd.DataFrame(imp_median.fit_transform(X_train_cat), columns = X_train_cat.columns)
X_train_cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 978 entries, 0 to 977
Data columns (total 41 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   MSZoning       978 non-null    object
 1   Street         978 non-null    object
 2   LotShape       978 non-null    object
 3   LandContour    978 non-null    object
 4   Utilities      978 non-null    object
 5   LotConfig      978 non-null    object
 6   LandSlope      978 non-null    object
 7   Neighborhood   978 non-null    object
 8   Condition1     978 non-null    object
 9   Condition2     978 non-null    object
 10  BldgType       978 non-null    object
 11  HouseStyle     978 non-null    object
 12  RoofStyle      978 non-null    object
 13  RoofMatl       978 non-null    object
 14  Exterior1st    978 non-null    object
 15  Exterior2nd    978 non-null    object
 16  MasVnrType     978 non-null    object
 17  ExterQual      978 non-null    object
 18  ExterCond      978 non-null   

In [20]:
X_train_cat.head()

,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,SaleType,SaleCondition,YearBuilt,GarageYrBlt,YrSold
0,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,Gd,TA,PConc,Gd,TA,No,GLQ,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,Attchd,RFn,TA,TA,Y,WD,Normal,2003,2003.0,2008
1,RL,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,Gable,CompShg,MetalSd,MetalSd,None,TA,TA,CBlock,Gd,TA,Gd,ALQ,Unf,GasA,Ex,Y,SBrkr,TA,Typ,Attchd,RFn,TA,TA,Y,WD,Normal,1976,1976.0,2007
2,RL,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,Gd,TA,PConc,Gd,TA,Mn,GLQ,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,Attchd,RFn,TA,TA,Y,WD,Normal,2001,2001.0,2008
3,RL,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,Gable,CompShg,Wd Sdng,Wd Shng,None,TA,TA,BrkTil,TA,Gd,No,ALQ,Unf,GasA,Gd,Y,SBrkr,Gd,Typ,Detchd,Unf,TA,TA,Y,WD,Abnorml,1915,1998.0,2006
4,RL,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,Gd,TA,PConc,Gd,TA,Av,GLQ,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,Attchd,RFn,TA,TA,Y,WD,Normal,2000,2000.0,2008


In [21]:
X_train_cat.shape

(978, 41)

In [22]:
## columns to be one hot encoded
dummeies_cat_cols = ['MSZoning', 'Street', 'LandContour', 'Utilities','LotConfig', 'BldgType',\
                     'HouseStyle','RoofStyle','RoofMatl','MasVnrType','Foundation','Heating',\
                     'CentralAir','Electrical','GarageType','PavedDrive','SaleCondition']

## column to be encoded orinally
ordinal_cat_cols = ['LotShape', 'LotConfig', 'ExterQual', 'ExterCond', 'BsmtQual','BsmtCond',\
                    'BsmtExposure','BsmtFinType1','BsmtFinType2','HeatingQC','KitchenQual','Functional',\
                    'GarageFinish','GarageQual','GarageCond','YearBuilt','GarageYrBlt','YrSold']

## I choose to remove these column as they add sprasity and make the model complex
high_cardinal_cols = ['Neighborhood', 'Condition1', 'Condition2', 'Exterior1st',\
                      'Exterior2nd', 'SaleType']

In [23]:
print(len(dummeies_cat_cols), len(ordinal_cat_cols), len(high_cardinal_cols))

17 18 6


In [24]:
## dropping columns of high cardinality
X_train_cat = X_train_cat.drop(labels=high_cardinal_cols, axis=1)
X_train_cat.shape

(978, 35)

## Ordinal encoding

In [25]:
## performing ordinal and one hot encoding
enc = OrdinalEncoder()
X_train_cat_ordinal = pd.DataFrame(enc.fit_transform(X_train_cat[ordinal_cat_cols]), columns = X_train_cat[ordinal_cat_cols].columns)
X_train_cat_ordinal.head()

,LotShape,LotConfig,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,HeatingQC,KitchenQual,Functional,GarageFinish,GarageQual,GarageCond,YearBuilt,GarageYrBlt,YrSold
0,3.0,4.0,2.0,4.0,2.0,3.0,3.0,2.0,5.0,0.0,2.0,6.0,1.0,4.0,4.0,98.0,86.0,2.0
1,3.0,2.0,3.0,4.0,2.0,3.0,1.0,0.0,5.0,0.0,3.0,6.0,1.0,4.0,4.0,71.0,59.0,1.0
2,0.0,4.0,2.0,4.0,2.0,3.0,2.0,2.0,5.0,0.0,2.0,6.0,1.0,4.0,4.0,96.0,84.0,2.0
3,0.0,0.0,3.0,4.0,3.0,1.0,3.0,0.0,5.0,2.0,2.0,6.0,2.0,4.0,4.0,14.0,81.0,0.0
4,0.0,2.0,2.0,4.0,2.0,3.0,0.0,2.0,5.0,0.0,2.0,6.0,1.0,4.0,4.0,95.0,83.0,2.0


## One hot encoding

In [26]:
# Convert into dummies
X_train_dummies = pd.get_dummies(X_train_cat[dummeies_cat_cols], drop_first=True)
X_train_dummies.head()

,MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Pave,LandContour_HLS,LandContour_Low,LandContour_Lvl,Utilities_NoSeWa,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_CompShg,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,CentralAir_Y,Electrical_FuseF,Electrical_FuseP,Electrical_SBrkr,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,PavedDrive_P,PavedDrive_Y,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,0,0,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,1,0
1,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,1,0
2,0,0,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,1,0
3,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0
4,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,1,0


In [27]:
X_train_dummies.shape

(978, 64)

##Mergeing the dataframes

In [28]:
# Merging the dataframes
X_train = pd.concat([X_train_numberic_scaled, X_train_dummies,X_train_cat_ordinal], axis=1)
X_train.head()

,MSSubClass,LotArea,OverallQual,OverallCond,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Pave,LandContour_HLS,LandContour_Low,LandContour_Lvl,Utilities_NoSeWa,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_CompShg,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,CentralAir_Y,Electrical_FuseF,Electrical_FuseP,Electrical_SBrkr,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,PavedDrive_P,PavedDrive_Y,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,LotShape,LotConfig,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,HeatingQC,KitchenQual,Functional,GarageFinish,GarageQual,GarageCond,YearBuilt,GarageYrBlt,YrSold
0,0.382353,0.035056,0.555556,0.75,0.216667,0.135188,0.000000,0.074058,0.153191,0.165213,0.0,0.0,0.135644,0.333333,0.0,0.333333,0.0,0.375,0.5,0.333333,0.000000,0.50,0.338505,0.140023,0.000000,0.0,0.0,0.0,0.0,0.0,0.363636,0,0,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,1,0,3.0,4.0,2.0,4.0,2.0,3.0,3.0,2.0,5.0,0.0,2.0,6.0,1.0,4.0,4.0,98.0,86.0,2.0
1,0.000000,0.033195,0.444444,0.50,0.950000,0.036499,0.000000,0.391267,0.183306,0.180358,0.0,0.0,0.148078,0.000000,0.0,0.333333,0.0,0.375,0.5,0.333333,0.000000,0.00,0.000000,0.000000,0.054845,0.0,0.0,0.0,0.0,0.0,1.000000,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,1,0,3.0,2.0,3.0,4.0,2.0,3.0,1.0,0.0,5.0,0.0,3.0,6.0,1.0,4.0,4.0,71.0,59.0,1.0
2,0.000000,0.034532,0.666667,0.50,0.916667,0.000000,0.000000,0.691781,0.264484,0.294172,0.0,0.0,0.241522,0.000000,0.0,0.666667,0.0,0.375,0.5,0.416667,0.000000,0.75,0.588152,0.242707,0.107861,0.0,0.0,0.0,0.0,0.0,0.272727,0,0,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,1,0,0.0,4.0,2.0,4.0,2.0,3.0,2.0,2.0,5.0,0.0,2.0,6.0,1.0,4.0,4.0,96.0,84.0,2.0
3,0.000000,0.042015,0.444444,0.75,0.250000,0.148122,0.000000,0.101455,0.175614,0.169573,0.0,0.0,0.139224,0.333333,0.0,0.333333,0.5,0.375,0.5,0.333333,0.000000,0.50,0.363188,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.454545,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0.0,0.0,3.0,4.0,3.0,1.0,3.0,0.0,5.0,2.0,2.0,6.0,2.0,4.0,4.0,14.0,81.0,0.0
4,0.000000,0.040711,0.444444,0.50,0.416667,0.189759,0.109139,0.083476,0.227332,0.242084,0.0,0.0,0.198757,0.333333,0.0,0.333333,0.0,0.250,0.5,0.333333,0.333333,0.50,0.294781,0.280047,0.069470,0.0,0.0,0.0,0.0,0.0,0.272727,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,1,0,0.0,2.0,2.0,4.0,2.0,3.0,0.0,2.0,5.0,0.0,2.0,6.0,1.0,4.0,4.0,95.0,83.0,2.0


In [29]:
X_train.shape

(978, 113)

# Perparing the test data

In [ ]:
X_test.info()

In [ ]:
## seprating numberical and categorical columns
X_test_numberic = X_test.select_dtypes(include=['number'])
X_test_cat = X_test.select_dtypes(include=['object'])

## Observing few column can be classifed as catergorical instead on numberical
cat_cols = ['YearBuilt', 'GarageYrBlt', 'YrSold']
X_test_numberic[cat_cols] = X_test_numberic[cat_cols].astype(object)
X_test_cat = pd.concat([X_test_cat.sort_index(), X_test_numberic[cat_cols].sort_index()],  axis=1)
X_test_numberic = X_test_numberic.drop(cat_cols, axis=1)
# print(X_test_cat.shape, X_test_numberic.shape)

## Imputing numberical cols
X_test_numberic = X_test_numberic.drop(labels=['LotFrontage','MasVnrArea'], axis=1)
# X_test_numberic.head()

# scaling the data between 0 and 1
X_test_numberic_scaled = pd.DataFrame(scaler.transform(X_test_numberic), columns = X_test_numberic.columns)

X_test_cat = X_test_cat.where(pd.notnull(X_test_cat), None)
imp_median = SimpleImputer(missing_values=None, strategy='most_frequent')
X_test_cat = pd.DataFrame(imp_median.fit_transform(X_test_cat), columns = X_test_cat.columns)
# X_test_cat.info()

X_test_cat = X_test_cat.drop(labels=high_cardinal_cols, axis=1)
# X_test_cat.shape

## performing ordinal and one hot encoding
enc = OrdinalEncoder()
X_test_cat_ordinal = pd.DataFrame(enc.fit_transform(X_test_cat[ordinal_cat_cols]), columns = X_test_cat[ordinal_cat_cols].columns)
# X_test_cat_ordinal.head()

# Convert into dummies
X_test_dummies = pd.get_dummies(X_test_cat[dummeies_cat_cols], drop_first=True)
# X_test_dummies.head()

# Merging the dataframes
X_test = pd.concat([X_test_numberic_scaled, X_test_dummies,X_test_cat_ordinal], axis=1)
X_test.head()

# Implementing RFE

In [35]:
regressor = RandomForestRegressor(n_estimators=100, max_depth=10)

#===========================================================================
# perform a scikit-learn Recursive Feature Elimination (RFE)
#===========================================================================
# here we want only one final feature, we do this to produce a ranking
n_features_to_select = 1
rfe = RFE(regressor, n_features_to_select=n_features_to_select)
rfe.fit(X_train, y_train)


#===========================================================================
# now print out the features in order of ranking
#===========================================================================
feature_list = []
from operator import itemgetter
features = X_train.columns.to_list()
for x, y in (sorted(zip(rfe.ranking_ , features), key=itemgetter(0))):
    print(x, y)
    feature_list.append(y)

top_15_numberic_features = X_train[feature_list[:15]]

top_15_numberic_features.head()

,OverallQual,GrLivArea,TotalBsmtSF,BsmtFinSF1,GarageArea,1stFlrSF,LotArea,YearRemodAdd,GarageCars,2ndFlrSF,WoodDeckSF,OpenPorchSF,TotRmsAbvGrd,BsmtUnfSF,MoSold
0,0.555556,0.135644,0.153191,0.135188,0.338505,0.165213,0.035056,0.216667,0.50,0.0,0.140023,0.000000,0.333333,0.074058,0.363636
1,0.444444,0.148078,0.183306,0.036499,0.000000,0.180358,0.033195,0.950000,0.00,0.0,0.000000,0.054845,0.333333,0.391267,1.000000
2,0.666667,0.241522,0.264484,0.000000,0.588152,0.294172,0.034532,0.916667,0.75,0.0,0.242707,0.107861,0.416667,0.691781,0.272727
3,0.444444,0.139224,0.175614,0.148122,0.363188,0.169573,0.042015,0.250000,0.50,0.0,0.000000,0.000000,0.333333,0.101455,0.454545
4,0.444444,0.198757,0.227332,0.189759,0.294781,0.242084,0.040711,0.416667,0.50,0.0,0.280047,0.069470,0.333333,0.083476,0.272727


In [36]:
top_10_numberic_features.shape

(978, 15)

# Linear Regression

In [31]:
def get_error_stats(y,y_pred,name="Test",DoPrint=True):
    MSE = mean_squared_error(y, y_pred)
    MAE = mean_absolute_error(y, y_pred)
    R2 = r2_score(y, y_pred)
    if DoPrint:
        print(F"MSE {name} {MSE}")
        print(F"MAE Score {name} {MAE}")
        print(F"R2 Score {name} {R2} \n")
    return MSE,MAE,R2

KeyboardInterrupt: ignored